# Basalt Prompt SDK Demo

This notebook demonstrates how to use the Basalt Prompt SDK to interact with your prompts, including:
- Retrieving prompts with tags and versions
- Using variables in prompts
- Integration with AI providers (OpenAI)
- Telemetry and observability patterns

This notebook uses primarily mock data for easy execution without API keys.

In [ ]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))  # Needed to make notebook work in VSCode

os.environ["BASALT_BUILD"] = "development"

from basalt import Basalt, TelemetryConfig

# Initialize the SDK with telemetry enabled
# Note: Replace with your actual API key for real usage
telemetry = TelemetryConfig(
    service_name="prompt-demo",
    environment="development",
    enable_llm_instrumentation=True,
)

basalt = Basalt(
    api_key=os.getenv("BASALT_API_KEY", "sk-demo-key"),  # Replace with your API key
    telemetry_config=telemetry,
    trace_user={"id": "demo-user"},
)

## 1. Getting a Prompt

Retrieve a specific prompt using a slug, with optional filters for tag and version.

In [ ]:
# Get a prompt by slug (default is production version)
# For real usage, replace 'prompt-slug' with your actual prompt slug
try:
    result = basalt.prompts.get_sync('prompt-slug')
except Exception:
    pass
    # In production, handle the error appropriately

## 2. Using Tags and Versions

You can specify a tag or version when retrieving a prompt.

In [ ]:
# Get a prompt with a specific tag
try:
    result_tag = basalt.prompts.get_sync(slug='prompt-slug', tag='latest')
except Exception:
    pass

# Get a prompt with a specific version
try:
    result_version = basalt.prompts.get_sync(slug='prompt-slug', version='1.0.0')
except Exception:
    pass

## 3. Using Variables

If your prompt has variables, you can pass them when fetching the prompt.

In [ ]:
# Get a prompt with variables
# Variables allow you to customize prompts with dynamic values
try:
    result_vars = basalt.prompts.get_sync(
        slug='prompt-slug-with-vars',
        variables={
            'name': 'John Doe',
            'role': 'Developer',
            'company': 'Acme Inc'
        }
    )
except Exception:
    pass

## 4. Using Prompts with AI Providers

Once you have retrieved a prompt, you can use it with your AI provider of choice.

In [ ]:
# Example with OpenAI (you'll need to install the openai package)
# This demonstrates how to use Basalt prompts with LLM providers
try:
    import openai

    # Set up OpenAI client (replace with your actual API key for real usage)
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY", "sk-demo-key"))

    # Get a prompt from Basalt
    try:
        result = basalt.prompts.get_sync('prompt-slug')

        # Use the prompt with OpenAI
        # Note: This will fail with demo keys, but shows the pattern
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": result.text}
            ]
        )
    except Exception:
        pass

except ImportError:
    pass

## 5. Error Handling

Proper error handling when working with prompts.

In [ ]:
# Example of handling different error scenarios
def get_prompt_safely(slug, tag=None, version=None, variables=None):
    """
    Helper function to safely retrieve prompts with proper error handling.

    Returns the prompt text on success, or None on failure.
    """
    try:
        result = basalt.prompts.get_sync(
            slug=slug,
            tag=tag,
            version=version,
            variables=variables
        )
        return result.text
    except Exception:
        return None

# Test with a non-existent prompt (will fail)
prompt_text = get_prompt_safely("non-existent-prompt")
if prompt_text:
    pass
else:
    pass

# Test with mock valid prompt (will also fail with demo keys)
prompt_text = get_prompt_safely("prompt-slug")
if prompt_text:
    pass
else:
    pass

## 6. Integration with Telemetry and Observability

You can use the Prompt SDK together with Basalt's observability features to trace prompt usage and LLM calls.

In [ ]:
from basalt.observability.context_managers import trace_generation, trace_span
from basalt.observability.decorators import evaluator


# Example: Trace a complete prompt-to-LLM workflow
@evaluator(
    slugs=["prompt-quality", "response-accuracy"],
    sample_rate=1.0,
    metadata=lambda prompt_slug, **kwargs: {
        "prompt_slug": prompt_slug,
        "source": "basalt_prompts"
    }
)
def generate_response_with_prompt(prompt_slug: str, variables: dict = None) -> str:
    """
    Fetch a prompt and use it to generate a response, with full observability.

    The @evaluator decorator will attach evaluators to any instrumented spans
    created within this function's context.
    """
    with trace_span("prompt_workflow") as span:
        span.set_input({"prompt_slug": prompt_slug, "variables": variables})

        # Step 1: Fetch prompt from Basalt
        try:
            prompt_result = basalt.prompts.get_sync(slug=prompt_slug, variables=variables)
            span.add_event("prompt_fetched")
            span.set_attribute("prompt.version", prompt_result.version)

            # Step 2: Use prompt with LLM (mocked here)
            with trace_generation("llm.generate_response") as llm_span:
                llm_span.set_model("gpt-4")
                llm_span.set_prompt(prompt_result.text)

                # Mock LLM response
                mock_response = f"Mock response using prompt: {prompt_result.slug}"

                llm_span.set_completion(mock_response)
                llm_span.set_output({"response": mock_response})

            span.set_output({"status": "success", "response": mock_response})
            return mock_response

        except Exception as e:
            span.record_exception(e)
            span.set_output({"status": "error", "error": str(e)})
            return f"Error: {e}"

# Execute the workflow
try:
    response = generate_response_with_prompt(
        prompt_slug="example-prompt",
        variables={"context": "demo"}
    )
except Exception:
    pass

# Don't forget to shutdown to flush traces
# basalt.shutdown()  # Uncomment when done with all operations